In [32]:
import pandas as pd
from datetime import datetime
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from nltk.corpus import stopwords


# Loading data from the linked repo

In [3]:
raw_data = pd.read_csv("../data/final_dank.csv")


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3,4,5,8,9,10,11,14,16,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Selecting relevant attributes

In [4]:
l = list(raw_data.columns.values)

l1 = l[4:26]
data = raw_data[l1]
print(data.columns.values)
data1 = data.drop(columns=['is_nsfw','media','thumbnail.height', 'thumbnail.thumbnail', 'thumbnail.width','time_of_day', 'datetime_temp', 'id', 'subreddit', 'url', 'created_utc', 'subscribers', 'downs', 'awards'])
print("-----------------")
print(data1.columns.values)

['awards' 'processed_words' 'created_utc' 'downs' 'id' 'is_nsfw' 'media'
 'subreddit' 'subscribers' 'thumbnail.height' 'thumbnail.thumbnail'
 'thumbnail.width' 'title' 'ups' 'url' 'datetime_temp' 'time_of_day'
 'ups_normed' 'dank_level' 'TextLength' 'Sentiment' 'word_count']
-----------------
['processed_words' 'title' 'ups' 'ups_normed' 'dank_level' 'TextLength'
 'Sentiment' 'word_count']


In [5]:
data1['dank_level'].unique()
# ezzel ez lesz a célváltozónk, bináris osztályozás esetén

array([ 0.,  1., nan])

In [8]:
# filtering where label is NA
data_useful = data1.loc[data1['dank_level'].notna()]
data_useful

,processed_words,title,ups,ups_normed,dank_level,TextLength,Sentiment,word_count
0,"['sometimesth', 'face', 'go', 'work', 'get', '...",It just be that way sometimes,1.0,0.000001,0.0,118.0,0.58,8.0
1,"['time', 'come', 'barter', 'pandem', 'day', 'r...",The time has come for the barter of pandemic d...,27.0,0.000036,0.0,93.0,0.29,8.0
2,"['gang', 'derin', 'mlke', 'wey', 'stealin', 's...",Day 10 of quarantine,1.0,0.000001,0.0,114.0,0.43,8.0
3,"['desper', 'time', 'desper', 'hoard', 'toilet'...","Desperate times, Desperate solutions",1.0,0.000001,0.0,97.0,0.18,8.0
4,"['desper', 'time', 'desper', 'hoard', 'toilet'...","Desperate times, Desperate solutions",1.0,0.000001,0.0,97.0,0.18,8.0
...,...,...,...,...,...,...,...,...
69937,"['stop', 'act', 'like', 'onegover', 'work', 'h...",stop acting like one,1.0,0.000021,0.0,55.0,0.43,7.0
69938,"['go', 'right', 'choos', 'fighter']",im going with the top right what about you?,1.0,0.000021,0.0,62.0,0.49,4.0
69939,"['pound', 'multi', 'million', 'dollar', 'compa...",SAM POUND!,1.0,0.000021,0.0,98.0,0.59,8.0
69940,"['second', 'virus', 'kill']",My Second one,1.0,0.000021,0.0,56.0,0.42,3.0


In [28]:
# testing some nltk
tester = data_useful['title'][1]
print(tester)
# remove punctuation
punctless = re.sub('[^a-zA-Z]', ' ', tester)
print(punctless)
# lowercase
lowercase = punctless.lower()
print(lowercase)
# stopword removal
words_list = lowercase.split()
words_list =  [word for word in words_list if not word in set(stopwords.words('english'))]
print(words_list)

The time has come for the barter of pandemic days.
The time has come for the barter of pandemic days 
the time has come for the barter of pandemic days 
['time', 'come', 'barter', 'pandemic', 'days']


In [35]:
# first try at stemming
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("english")
word_stem = []
for word in words_list:
    word_stem.append(stemmer.stem(word))
print(word_stem)

['time', 'come', 'barter', 'pandem', 'day']


In [40]:
# the processed words are not lemmatized as far as i know, will need to look into that
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

